In [1]:
import base64
import requests

import IPython.display
import ipywidgets

prompt = ipywidgets.widgets.Textarea(
    value="White Dog",
    description="Prompt: ",
)

model = ipywidgets.widgets.Dropdown(
    options=["dall-e-2", "dall-e-3"],
    value="dall-e-3",
    description="Model: ",
)

n = ipywidgets.widgets.IntSlider(
    min=1,
    max=10,
    step=1,
    value=1,
    description="Number: ",
)

quality = ipywidgets.widgets.Dropdown(
    options=["standard", "hd"],
    value="hd",
    description="Quality: ",
)

size = ipywidgets.widgets.Dropdown(
    options=["256x256", "512x512", "1024x1024", "1792x1024", "1024x1792"],
    value="1024x1024",
    description="Size: ",
)

style = ipywidgets.widgets.Dropdown(
    options=["vivid", "natural"],
    value="vivid",
    description="Style: ",
)

output = ipywidgets.widgets.Output()

button = ipywidgets.widgets.Button(
    description="Generate",
)


@output.capture()
def generate(b: ipywidgets.widgets.Button):
    b.disabled = True

    r = requests.post("http://cortex-api.cortex-api.svc.cluster.local:8080/v1/images/generations", json={
        "prompt": prompt.value,
        "model": model.value,
        "n": n.value,
        "quality": quality.value,
        "response_format": "b64_json",
        "size": size.value,
        "style": style.value,
    }, headers={
        "Content-Type": "application/json"
    })

    response = r.json()

    if "error" in response:
        import sys
        print(response["error"]["message"], file=sys.stderr)
        sys.exit(1)

    images = []
    for image in response.get("data", []):
        if "revised_prompt" in image:
            print(image["revised_prompt"])
        data = base64.b64decode(image["b64_json"])
        images.append(IPython.display.Image(data=data))

    IPython.display.display(*images)

    b.disabled = False


button.on_click(generate)

IPython.display.display(prompt, model, n, quality, size, style, button, output)

Text(value='White Dog', description='Prompt: ')

Dropdown(description='Model: ', index=1, options=('dall-e-2', 'dall-e-3'), value='dall-e-3')

IntSlider(value=1, description='Number: ', max=10, min=1)

Dropdown(description='Quality: ', index=1, options=('standard', 'hd'), value='hd')

Dropdown(description='Size: ', index=2, options=('256x256', '512x512', '1024x1024', '1792x1024', '1024x1792'),…

Dropdown(description='Style: ', options=('vivid', 'natural'), value='vivid')

Button(description='Generate', style=ButtonStyle())

Output()